### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [2]:
import sys
from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import predict_count
from countpassenger import Dataset

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

df_with_count_tmp = pd.DataFrame()
CAMERA_LIST = ["mbk-14-11", "mbk-14-12", "mbk-14-13", "mbk-14-14"]
# current_cam = "mbk-14-14"

# SAVE_DIR = os.path.join(
#     conf.RESOURCES_PROCESSED_DIR, f"2024-07-01/count-passengers-{current_cam}-vehicle.csv"
# )

load_param = dict(
    vehicle_csv_path="/home/jeans/internship/parking-customer-count/resources/raw/2024-07-01/mbk-tourist-vehicle-object-20240701-20240731.csv",
    cross_csv_path="/home/jeans/internship/parking-customer-count/resources/raw/2024-07-01/mbk-tourist-raw-cross-object-20240701-20240731.csv",
    reverse_csv_path="/home/jeans/internship/parking-customer-count/resources/raw/2024-07-01/mbk-tourist-raw-reverse-object-20240701-20240731.csv",
)
DEBUG = False

for current_cam in CAMERA_LIST:
    # load the dataset from directory path
    df_vehicle_raw, df_cross_raw, df_reverse_raw = Dataset.load_dataset_from_paths(**load_param)

    # preprocess the data
    df_vehicle = Preprocess.df_clean_vehicle(df_vehicle_raw=df_vehicle_raw, current_camera=current_cam)
    df_cross = Preprocess.df_clean_customer(df_cross_raw, current_cam=current_cam)
    df_reverse = Preprocess.df_clean_customer(df_reverse_raw, current_cam=current_cam)

    if DEBUG:
        print("-" * 60)
        print("VEHICLE DATASET")
        print(
            df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT, "timestamp_unix", "timestamp_unix_end"]]
            .head()
            .to_string()
        )
        print("-" * 60)
        print("CROSS DATASET")
        print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].head().to_string())
        print("-" * 60)
        print("REVERSE DATASET")
        print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].head().to_string())

    df_vehicle_with_cross_and_reverse = predict_count(
        df_vehicle,
        df_cross,
        df_reverse,
    )
    
    print(f"processing for cam:{current_cam} completed. total entry: {df_vehicle_with_cross_and_reverse.shape[0]}")
    df_with_count_tmp = pd.concat([df_with_count_tmp, df_vehicle_with_cross_and_reverse], ignore_index=True)
    
print(df_with_count_tmp.shape)

(4871, 36)
(4871, 36)


KeyboardInterrupt: 

## TOGETHER


In [ ]:
# df_reverse_with_reverse_count = match_reverse_to_vehicle(df_reverse, df_vehicle)
# df_vehicle_with_cross_and_reverse = match_cross_to_vehicle(
#     df_cross,
#     df_reverse_with_reverse_count,
# )

In [ ]:
df_vehicle_with_cross_and_reverse.to_csv(path_or_buf=SAVE_DIR, index=False)